In [1]:
!pip install ts2vg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00


In [14]:
import pandas as pd
import numpy as np
import seaborn as sns

from ts2vg import *

In [6]:
shop_df = pd.read_csv('/content/release_10_23_2020.csv')
shop_df.head()

,session_id_hash,event_type,product_action,product_skus_hash,server_timestamp_epoch_ms,hashed_url
0,00000005c19449b57d8d05dfc8b5474be0654032,pageview,NaN,NaN,1544469996111,da99729886aff70a02733b6cd69ee7df35622d9302347e...
1,00006a0ada94a5186163a25e9ed9c94481c820d9,pageview,NaN,NaN,1545737959865,e2f7e0cee4272e804f0d323a3513dd01716a5a40ab9abf...
2,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908835402,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
3,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908847263,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
4,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,detail,90add4dd072e160034e98ddea257e0b59441eae00d8955...,1544909035879,8fa1ecf31ececb27ebe9c529966f3d1f907542fe138d5d...


In [7]:
print(shop_df.event_type.value_counts())
print(shop_df.product_action.value_counts())

pageview    413334
event        83339
Name: event_type, dtype: int64
detail      148918
add          72068
click         6352
remove        4622
purchase       942
Name: product_action, dtype: int64


In [8]:
print(shop_df['product_action'].unique())
shop_df.drop(columns=['event_type', 'product_skus_hash', 'hashed_url'], axis=1, inplace=True)
shop_df.head()

[nan 'detail' 'click' 'add' 'purchase' 'remove']


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,NaN,1544469996111
1,00006a0ada94a5186163a25e9ed9c94481c820d9,NaN,1545737959865
2,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908835402
3,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908847263
4,00007d15aeb741b3cdd873cb3933351d699cc320,detail,1544909035879


In [9]:
shop_df['product_action'].replace(np.nan,1, inplace=True)
symbols = {
    'detail': 2,
    'add': 3,
    'remove': 4,
    'purchase': 5,
    'click': 6
}
shop_df['product_action'].replace(symbols, inplace=True)

In [10]:
shop_df = shop_df.groupby(shop_df.session_id_hash).agg(list).reset_index()
print(shop_df.shape)
shop_df.head()

(40283, 3)


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,[1],[1544469996111]
1,00006a0ada94a5186163a25e9ed9c94481c820d9,[1],[1545737959865]
2,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]","[1544908835402, 1544908847263, 1544909035879, ..."
3,0000c49d1359c87f112285d63a9fc0d4c50f73b4,"[1, 1, 1]","[1544780629084, 1544780652666, 1544780882467]"
4,0001135282dbaf0e734345212ea6f8da9f5b66ed,[1],[1544422592886]


In [11]:
# # Truncating till 5
shop_df['conversion_class'] = shop_df['product_action'].apply(lambda x: 'C' if 5 in x else 'NC')
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x[0:x.index(5)] if 5 in x else x)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) >= 8 else np.nan)
shop_df.dropna(inplace=True)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) <= 155 else np.nan)
shop_df.dropna(inplace=True)
print(shop_df.shape)
print(shop_df.conversion_class.value_counts())
shop_df.head()  

(13764, 4)
NC    13063
C       701
Name: conversion_class, dtype: int64


,session_id_hash,product_action,server_timestamp_epoch_ms,conversion_class
5,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]","[1545550515947, 1545550562911, 1545550685332, ...",NC
13,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]","[1545133756068, 1545133781992, 1545133945223, ...",NC
14,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]","[1544880442310, 1544880482619, 1544880489778, ...",NC
17,00037198b9e159451e6947af787d7325133e0d93,"[1, 1, 2, 1, 2, 1, 2, 1, 2, 1]","[1545720158541, 1545720163904, 1545720181542, ...",NC
18,0003ab40573f98a5e4d9da2d00ed311f367ef71f,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...","[1544728636006, 1544728790217, 1544728840881, ...",NC


In [12]:
def reduced_time(time):
    index = 0
    for sec in time:
        if index<len(time)-1:                           # do -2 when using all events
            time[index] = abs(time[index+1] - time[index])
        index+=1
    time.pop()

    # final_val = time.pop()
    # final_val = abs(final_val - avg)
    # time.append(final_val)
    
    total_time = sum(time)
    index = 0
    for sec in time:
        time[index] = sec/total_time
        index+=1
    
    return time

shop_preprocessed_df = shop_df.copy()    
shop_preprocessed_df['reduced_time'] = shop_preprocessed_df['server_timestamp_epoch_ms'].apply(lambda x: reduced_time(x))

In [13]:
shop_preprocessed_df.reset_index(inplace=True)
shop_preprocessed_df.drop(['server_timestamp_epoch_ms', 'index'], axis=1, inplace=True)
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time
0,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0..."
1,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0...."
2,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0..."
3,00037198b9e159451e6947af787d7325133e0d93,"[1, 1, 2, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.036022057885156604, 0.11847045627044418, 0...."
4,0003ab40573f98a5e4d9da2d00ed311f367ef71f,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",NC,"[0.0646781714579064, 0.021249164318650225, 0.0..."


In [18]:
def generate_edges(time_list):
    vg = HorizontalVG()
    vg.build(time_list)
    edges = vg.edges
    return edges

shop_preprocessed_df['HVGms_edges'] = shop_preprocessed_df['reduced_time'].apply(lambda x: generate_edges(x))
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time,HVGms_edges
0,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","[(10, 11), (11, 12), (11, 13), (11, 14), (9, 1..."
1,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","[(5, 6), (3, 6), (1, 6), (0, 1), (1, 2), (1, 3..."
2,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","[(9, 10), (8, 10), (2, 10), (1, 2), (0, 2), (2..."
3,00037198b9e159451e6947af787d7325133e0d93,"[1, 1, 2, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.036022057885156604, 0.11847045627044418, 0....","[(3, 4), (1, 4), (4, 5), (4, 6), (0, 1), (1, 2..."
4,0003ab40573f98a5e4d9da2d00ed311f367ef71f,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",NC,"[0.0646781714579064, 0.021249164318650225, 0.0...","[(23, 24), (21, 24), (20, 24), (19, 24), (14, ..."


In [22]:
def pattern_hvg_4(edges):  
    i = 0
    j = edges[len(edges)-1][1]
    pattern = list()
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+3]
        # print(i)
        if len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('F')
            elif [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('E')
        
        elif len(edges_tb_used) == 4:
            if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('D')
            elif [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('C')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('B')
            
        elif len(edges_tb_used) == 3:
            pattern.append('A')
        
        i+=1
                
    return pattern

In [23]:
def pattern_hvg_5(edges):
    i = 0
    j = edges[len(edges)-1][1]
    pattern = list()
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+4]
        # print(edges_tb_used)
        if len(edges_tb_used) == 7:
            if [(i, i+1), (i, i+2), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('P')
            else:
                pattern.append('O')
                
        elif len(edges_tb_used) == 6:
            if [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('N')
            elif [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('M')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('L')
            elif [(i, i+1), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('K')
            elif [(i, i+1), (i, i+2), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('J')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('I')
            elif [(i, i+1), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('H')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('G')    
            
        elif len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('F')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('E')
            elif [(i, i+1), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('D')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('C')
            else:
                pattern.append('B')
            
        elif len(edges_tb_used) == 4:
            pattern.append('A')
            
        i+=1
    return pattern

In [24]:
shop_preprocessed_df['pattern_hvg_4_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_4)
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time,HVGms_edges,pattern_hvg_4_nodes
0,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","[(10, 11), (11, 12), (11, 13), (11, 14), (9, 1...","[A, D, A, D]"
1,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","[(5, 6), (3, 6), (1, 6), (0, 1), (1, 2), (1, 3...","[D, C, D]"
2,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","[(9, 10), (8, 10), (2, 10), (1, 2), (0, 2), (2...","[D, C]"
3,00037198b9e159451e6947af787d7325133e0d93,"[1, 1, 2, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.036022057885156604, 0.11847045627044418, 0....","[(3, 4), (1, 4), (4, 5), (4, 6), (0, 1), (1, 2...","[D, A, D, C]"
4,0003ab40573f98a5e4d9da2d00ed311f367ef71f,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",NC,"[0.0646781714579064, 0.021249164318650225, 0.0...","[(23, 24), (21, 24), (20, 24), (19, 24), (14, ...","[A, A, A, D, C, A, A, D]"


In [25]:
shop_preprocessed_df['pattern_hvg_5_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_5)
shop_preprocessed_df.head()

,session_id_hash,product_action,conversion_class,reduced_time,HVGms_edges,pattern_hvg_4_nodes,pattern_hvg_5_nodes
0,00014c4428e61aca7e64be1064d2b6cf49233cfb,"[1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 1, 1, 1, 1]",NC,"[0.04219143180815674, 0.10998035246968647, 0.0...","[(10, 11), (11, 12), (11, 13), (11, 14), (9, 1...","[A, D, A, D]","[D, O, B, B]"
1,00033ce5ea7a3c31c62320c4631860fcf638f5b3,"[1, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.016090250390711797, 0.10131259302292382, 0....","[(5, 6), (3, 6), (1, 6), (0, 1), (1, 2), (1, 3...","[D, C, D]","[B, M]"
2,0003537cea83b232652aa2d29f90fec46e523443,"[1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]",NC,"[0.024222994086822748, 0.004302076823229652, 0...","[(9, 10), (8, 10), (2, 10), (1, 2), (0, 2), (2...","[D, C]","[B, O]"
3,00037198b9e159451e6947af787d7325133e0d93,"[1, 1, 2, 1, 2, 1, 2, 1, 2, 1]",NC,"[0.036022057885156604, 0.11847045627044418, 0....","[(3, 4), (1, 4), (4, 5), (4, 6), (0, 1), (1, 2...","[D, A, D, C]","[B, B, F]"
4,0003ab40573f98a5e4d9da2d00ed311f367ef71f,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",NC,"[0.0646781714579064, 0.021249164318650225, 0.0...","[(23, 24), (21, 24), (20, 24), (19, 24), (14, ...","[A, A, A, D, C, A, A, D]","[O, B, A, A, D, B, O, B, B, A, B, B]"


In [26]:
print(shop_preprocessed_df.loc[1, 'reduced_time'])
print(shop_preprocessed_df.loc[1, 'HVGms_edges'])

[0.016090250390711797, 0.10131259302292382, 0.005448862373864329, 0.022046200195883468, 0.00855469530686548, 0.009425495387800855, 0.8371219033219502]
[(5, 6), (3, 6), (1, 6), (0, 1), (1, 2), (1, 3), (2, 3), (3, 4), (3, 5), (4, 5)]


In [27]:
shop_preprocessed_df.to_csv('/content/shopper_data_with_hvgms_edges_pattern_ts2vg.csv', index=False)